## Logging on

Use Selenium to visit https://webapps1.chicago.gov/buildingrecords/ and accept the agreement.

> Think about when you use `.find_element...` and when you use `.find_elementSSS...`

In [64]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service as BraveService

from webdriver_manager.core.utils import ChromeType
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"

driver = webdriver.Chrome(options=options, service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))

In [65]:
import pandas as pd

In [66]:
driver.get('https://webapps1.chicago.gov/buildingrecords/')

In [67]:
# Accept agreement

driver.find_element(By.ID, "rbnAgreement1").click()

In [68]:
# Click submit on agreement

driver.find_element(By.ID, "submit").click()

## Searching

Search for **400 E 41ST ST**.

In [69]:
# Find the building address textbox and enter address

driver.find_element(By.ID, "fullAddress").send_keys("400 E 41ST ST.")

In [70]:
# Click submit button

driver.find_element(By.ID, "submit").click()

## Saving tables with pandas

Use pandas to save a CSV of all **permits** to `Permits - 400 E 41ST ST.csv`. Note that there are **different sections of the page**, not just one long permits table.

> - *Tip: When using `.read_html`, try using `flavor='lxml'` and comparing the results to `flavor='html5lib'`. Which works better?*
> - *Tip: You might need to install `html5lib` using `pip`. If so, you'll need to restart the notebook using **Kernel > Restart** before it will work.*

In [71]:
df = pd.read_html(driver.page_source, flavor='html5lib')[0]
df.head()

,PERMIT #,DATE ISSUED,DESCRIPTION OF WORK
0,100961712,03/09/2022,2017 Fire Service Upgrade Mandate On Five (5) ...
1,100845718,10/11/2019,ERECT TWO SCAFFOLDS FROM 10/14/2019 TO 10/14/2...
2,100778302,08/17/2018,PERMIT EXPIRES ON 10/17/2018 Erection Starts: ...
3,100721255,08/24/2017,PERMIT EXPIRES ON 10/24/2017 ERECTION STARTS: ...
4,100693399,03/03/2017,INSTALLATION OF LOW VOLTAGE BURGLAR ALARM INTE...


In [72]:
df.to_csv('Permits - 400 E 41ST ST.csv', index=False)

## Saving tables the long way

Save a CSV of all DOB inspections to `Inspections - 400 E 41ST ST.csv`.

This is more complicated than the last one becuse **we also need to save the URL to the inspection** (see how the inspection number is a link?). As a result, you won't be able to use pandas! Instead, you'll need to use a loop and create a list of dictionaries.

You can use Selenium or you can feed the source to BeautifulSoup. You should have approximately 160 rows.

You'll probably need to find the table first, then the rows inside, then the cells inside of each row. You'll probably use lots of list indexing. I might recommend XPath for finding the table.

*Tip: If you get a "list index out of range" error, it's probably due to an issue involving `thead` vs `tbody` elements. What are they? What are they for? What's in them? There are a few ways to troubleshoot it.*

In [73]:
inspections = driver.find_element(By.XPATH, '//*[@id="resultstable_inspections"]')

In [74]:
rows = inspections.find_elements(By.CSS_SELECTOR, 'tbody tr')

In [86]:
inspection_list = []

for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    
    number = columns[0].text
    url = columns[0].find_element(By.TAG_NAME, 'a').get_attribute('href')
    date = columns[1].text
    status = columns[2].text
    description = columns[3].text
    
    data = {
        'number': number,
        'url': url,
        'date': date,
        'status': status,
        'description': description
    }
    
    inspection_list.append(data)
    
df = pd.DataFrame(inspection_list)
df.head()

df.to_csv('Inspections - 400 E 41ST ST.csv', index=False)

### Bonus preview of Wednesday's content

**You prrrrrobably shouldn't do this one unless you want a real challenge.**

If you click the inspection number, it'll open up a new window that shows you details of the violations from that visit. Count the number of violations for each visit and save it in a new column called **num_violations**.

Save this file as `Inspections - 400 E 41ST ST - with counts.csv`.

Since clicking the link opens in a new window, we have to say "Hey Selenium, pay attention to that new window!" We do that with `driver.switch_to.window(driver.window_handles[-1])` (each window gets a `window_handle`, and we're just asking the driver to switch to the last one.). A rough sketch of what your code will look like is here:

```python
# Click the link that opens the new window

# Switch to the new window/tab
driver.switch_to.window(driver.window_handles[-1])

# Do your scraping in here

# Close the new window/tab
driver.close()

# Switch back to the original window/tab
driver.switch_to.window(driver.window_handles[0])
```

You'll want to play around with them individually before you try it with the whole set - the ones that pass are very different pages than the ones with violations! There are a few ways to get the number of violations, some easier than others.

In [89]:
inspection_list = []

for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    
    number = columns[0].text
    link = columns[0].find_element(By.TAG_NAME, 'a')
    url = columns[0].find_element(By.TAG_NAME, 'a').get_attribute('href')
    date = columns[1].text
    status = columns[2].text
    description = columns[3].text
    
    # Click the link for violation details
    link.click()
    # Switch driver to new window
    driver.switch_to.window(driver.window_handles[-1])
    
    # Look for a table
    # If found, count the number of violations
    # If not, return 0
    try:
        table = driver.find_element(By.CSS_SELECTOR, '#resultstable')
        violations = table.find_elements(By.CSS_SELECTOR, 'tbody tr')
        num_violations = len(violations)
    except:
        num_violations = 0

    driver.close()

    driver.switch_to.window(driver.window_handles[0])
    
    data = {
        'number': number,
        'url': url,
        'date': date,
        'status': status,
        'description': description,
        'num_violations': num_violations
    }
    
    inspection_list.append(data)
    
df = pd.DataFrame(inspection_list)
df.to_csv('Inspections - 400 E 41ST ST - with counts.csv', index=False)

df.head()

,number,url,date,status,description,num_violations
0,13821288,https://webapps1.chicago.gov/buildingrecords/i...,05/16/2022,PASSED,PERMIT INSPECTION,0
1,13813188,https://webapps1.chicago.gov/buildingrecords/i...,05/16/2022,PASSED,ANNUAL INSPECTION,0
2,13808208,https://webapps1.chicago.gov/buildingrecords/i...,05/13/2022,FAILED,PERMIT INSPECTION,1
3,13748085,https://webapps1.chicago.gov/buildingrecords/i...,04/25/2022,FAILED,ANNUAL INSPECTION,2
4,13684797,https://webapps1.chicago.gov/buildingrecords/i...,03/21/2022,FAILED,ANNUAL INSPECTION,2
